In [1]:
%pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error

# Load and preprocess the dataset (assuming you have a CSV file)
data = pd.read_csv("../data/diamonds.csv")
data.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [2]:
df = pd.read_csv("~/Downloads/rideshare_kaggle.csv")
df.head()

,id,timestamp,hour,day,month,datetime,timezone,source,destination,cab_type,...,precipIntensityMax,uvIndexTime,temperatureMin,temperatureMinTime,temperatureMax,temperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,apparentTemperatureMax,apparentTemperatureMaxTime
0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,1.544953e+09,9,16,12,2018-12-16 09:30:07,America/New_York,Haymarket Square,North Station,Lyft,...,0.1276,1544979600,39.89,1545012000,43.68,1544968800,33.73,1545012000,38.07,1544958000
1,4bd23055-6827-41c6-b23b-3c491f24e74d,1.543284e+09,2,27,11,2018-11-27 02:00:23,America/New_York,Haymarket Square,North Station,Lyft,...,0.1300,1543251600,40.49,1543233600,47.30,1543251600,36.20,1543291200,43.92,1543251600
2,981a3613-77af-4620-a42a-0c0866077d1e,1.543367e+09,1,28,11,2018-11-28 01:00:22,America/New_York,Haymarket Square,North Station,Lyft,...,0.1064,1543338000,35.36,1543377600,47.55,1543320000,31.04,1543377600,44.12,1543320000
3,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,1.543554e+09,4,30,11,2018-11-30 04:53:02,America/New_York,Haymarket Square,North Station,Lyft,...,0.0000,1543507200,34.67,1543550400,45.03,1543510800,30.30,1543550400,38.53,1543510800
4,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,1.543463e+09,3,29,11,2018-11-29 03:49:20,America/New_York,Haymarket Square,North Station,Lyft,...,0.0001,1543420800,33.10,1543402800,42.18,1543420800,29.11,1543392000,35.75,1543420800


In [21]:
df = df.sort_values(by=["timestamp"]).reset_index()
df.head()

,index,id,timestamp,hour,day,month,datetime,timezone,source,destination,...,precipIntensityMax,uvIndexTime,temperatureMin,temperatureMinTime,temperatureMax,temperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,apparentTemperatureMax,apparentTemperatureMaxTime
0,276564,90e22612-d1c7-47c2-af54-2a2240109889,1.543204e+09,3,26,11,2018-11-26 03:40:46,America/New_York,Beacon Hill,North End,...,0.1396,1543161600,40.61,1543122000,46.15,1543154400,38.23,1543136400,43.17,1543186800
1,30792,ef4771c2-c88d-4730-aaf7-a95751e9d27e,1.543204e+09,3,26,11,2018-11-26 03:40:46,America/New_York,Boston University,Theatre District,...,0.1396,1543161600,40.61,1543122000,46.15,1543154400,38.23,1543136400,43.17,1543186800
2,261362,d9852684-4e38-494c-ad6d-d8090e857ba9,1.543204e+09,3,26,11,2018-11-26 03:40:46,America/New_York,Beacon Hill,Boston University,...,0.1396,1543161600,40.61,1543122000,46.15,1543154400,38.23,1543136400,43.17,1543186800
3,449958,68c01519-3da8-4f84-b113-48cf6d1441db,1.543204e+09,3,26,11,2018-11-26 03:40:46,America/New_York,South Station,Beacon Hill,...,0.1396,1543161600,40.61,1543122000,46.15,1543154400,38.23,1543136400,43.17,1543186800
4,261363,f3f35cea-9a3f-4534-930d-da42e7610f77,1.543204e+09,3,26,11,2018-11-26 03:40:46,America/New_York,Beacon Hill,Boston University,...,0.1396,1543161600,40.61,1543122000,46.15,1543154400,38.23,1543136400,43.17,1543186800


In [68]:
df.loc[
    0:100,
    [
        "source",
        "destination",
        "cab_type",
        "name",
        "hour",
        "day",
        "month",
        "distance",
        "precipIntensity",
        # "surge_multiplier",
        "temperature",
        "pressure",
        "humidity",
        "price",
    ],
]

,source,destination,cab_type,name,hour,day,month,distance,precipIntensity,temperature,pressure,humidity,price
0,Beacon Hill,North End,Uber,Black,3,26,11,2.19,0.0,41.83,1014.10,0.92,17.5
1,Boston University,Theatre District,Lyft,Lux Black XL,3,26,11,3.03,0.0,41.83,1014.10,0.92,34.0
2,Beacon Hill,Boston University,Lyft,Shared,3,26,11,2.30,0.0,41.83,1014.10,0.92,3.5
3,South Station,Beacon Hill,Uber,Taxi,3,26,11,2.50,0.0,41.83,1014.10,0.92,NaN
4,Beacon Hill,Boston University,Lyft,Lux,3,26,11,2.30,0.0,41.83,1014.10,0.92,16.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,Theatre District,Boston University,Lyft,Shared,4,26,11,4.73,0.0,41.52,1014.06,0.90,11.0
97,Haymarket Square,North Station,Lyft,Lux,4,26,11,0.55,0.0,41.52,1014.06,0.90,10.5
98,South Station,Beacon Hill,Uber,UberPool,4,26,11,2.58,0.0,41.52,1014.06,0.90,8.0
99,West End,Northeastern University,Lyft,Lux Black XL,4,26,11,3.11,0.0,41.52,1014.06,0.90,34.0


0      17.5
1      34.0
2       3.5
4      16.5
5       7.0
       ... 
102    36.0
103    38.5
104     3.5
105    34.0
106    22.0
Name: price, Length: 100, dtype: float64

In [64]:
# Separate features and target variable
df2 = df.dropna()
df2 = df2.head(100)
X = df2.drop("price", axis=1)
y = df2["price"]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Preprocess categorical features
cat_columns = [
    "source",
    "destination",
    "cab_type",
    "name",
]
X_train_cat = X_train[cat_columns]
X_test_cat = X_test[cat_columns]

label_encoders = {}
for col in cat_columns:
    le = LabelEncoder()
    X_train_cat[col] = le.fit_transform(X_train_cat[col].copy())
    X_test_cat[col] = le.transform(X_test_cat[col].copy())
    label_encoders[col] = le

# Preprocess numeric features
num_columns = [
    "hour",
    "day",
    "month",
    "distance",
    "precipIntensity",
    # "surge_multiplier",
    "temperature",
    "pressure",
    "humidity",
]
scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train[num_columns])
X_test_num = scaler.transform(X_test[num_columns])

X_train_cat_tensor = torch.tensor(
    X_train_cat.values, dtype=torch.int64
)  # Use int64 dtype for categorical indices
X_train_num_tensor = torch.tensor(X_train_num, dtype=torch.float32)
X_test_cat_tensor = torch.tensor(
    X_test_cat.values, dtype=torch.int64
)  # Use int64 dtype for categorical indices
X_test_num_tensor = torch.tensor(X_test_num, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)


# Define the neural network model
class DiamondPricePredictor(nn.Module):
    def __init__(self, num_input_dim, cat_embedding_sizes, hidden_dim):
        super(DiamondPricePredictor, self).__init__()

        # Embedding layers for categorical features
        self.embeddings = nn.ModuleList(
            [
                nn.Embedding(num_classes, emb_size)
                for num_classes, emb_size in cat_embedding_sizes
            ]
        )

        total_emb_dim = sum(emb_size for _, emb_size in cat_embedding_sizes)
        self.fc1 = nn.Linear(num_input_dim + total_emb_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)

    def forward(self, num_inputs, cat_inputs):
        embeddings = [
            embedding(cat_inputs[:, i]) for i, embedding in enumerate(self.embeddings)
        ]
        cat_features = torch.cat(embeddings, dim=1)
        x = torch.cat([num_inputs, cat_features], dim=1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


num_input_dim = X_train_num.shape[1]
cat_embedding_sizes = [
    (len(le.classes_), min(50, (len(le.classes_) + 1) // 2))
    for le in label_encoders.values()
]
hidden_dim = 64
model = DiamondPricePredictor(num_input_dim, cat_embedding_sizes, hidden_dim)

/var/folders/rs/qflxwtyx6kvfj8jcqx5zm5hr0000gn/T/ipykernel_4807/1676805110.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_cat[col] = le.fit_transform(X_train_cat[col].copy())
/var/folders/rs/qflxwtyx6kvfj8jcqx5zm5hr0000gn/T/ipykernel_4807/1676805110.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_cat[col] = le.transform(X_test_cat[col].copy())
/var/folders/rs/qflxwtyx6kvfj8jcqx5zm5hr0000gn/T/ipykernel_4807/1676805110.py:25: SettingWithCopyWarning: 
A value is trying to be set on a 

ValueError: y contains previously unseen labels: 'South Station'

In [65]:
outputs = model(X_train_num_tensor, X_train_cat_tensor)
outputs

tensor([[-0.6209],
        [-0.1325],
        [-0.2881],
        [ 0.0968],
        [-0.0953],
        [-0.4319],
        [-0.1174],
        [-0.1601],
        [-0.3293],
        [ 0.3193],
        [-0.2006],
        [-0.0949],
        [-0.3470],
        [-0.4423],
        [-0.0903],
        [-0.0209],
        [ 0.1232],
        [-0.1612],
        [-0.0320],
        [-0.6046],
        [-0.3500],
        [-0.0740],
        [-0.4677],
        [-0.4586],
        [ 0.0316],
        [-0.0613],
        [-0.1205],
        [ 0.0625],
        [-0.1287],
        [-0.1334],
        [-0.2792],
        [-0.0185],
        [-0.1273],
        [-0.4536],
        [-0.1592],
        [ 0.3383],
        [-0.3883],
        [-0.0969],
        [-0.6642],
        [-0.1163],
        [-0.1587],
        [-0.1673],
        [-0.6315],
        [-0.0618],
        [-0.3102],
        [ 0.0627],
        [-0.2514],
        [-0.3541],
        [-0.6075],
        [-0.0817],
        [ 0.0839],
        [-0.6619],
        [-0.

In [66]:
outputs.shape

torch.Size([80, 1])

In [67]:
y_train_tensor

tensor([[10.0000],
        [19.5000],
        [26.0000],
        [19.5000],
        [10.5000],
        [19.5000],
        [10.5000],
        [11.0000],
        [13.0000],
        [10.5000],
        [ 9.0000],
        [27.5000],
        [10.5000],
        [ 8.5000],
        [10.5000],
        [ 7.0000],
        [14.0000],
        [27.5000],
        [34.0000],
        [    nan],
        [14.0000],
        [16.5000],
        [10.5000],
        [10.0000],
        [35.0000],
        [47.5000],
        [16.5000],
        [34.0000],
        [11.0000],
        [62.5000],
        [10.0000],
        [    nan],
        [10.5000],
        [16.0000],
        [18.5000],
        [38.5000],
        [ 7.0000],
        [30.0000],
        [27.5000],
        [ 9.0000],
        [10.5000],
        [34.0000],
        [ 9.5000],
        [22.5000],
        [16.0000],
        [13.5000],
        [ 7.0000],
        [ 9.0000],
        [    nan],
        [10.5000],
        [32.5000],
        [26.0000],
        [16.

In [47]:
loss = criterion(outputs, y_train_tensor)
loss

tensor(nan, grad_fn=<MseLossBackward0>)

In [29]:
# Define loss function and optimizer final:
# 616,079.6875 ->
# 261,726.6875
# 988,246.6875

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode="min",
    factor=0.8,
    patience=5,
    threshold=0.001,
    threshold_mode="rel",
    cooldown=0,
    min_lr=0.0001,
    eps=1e-08,
    verbose=False,
)

epochs = 1000
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(
        X_train_num_tensor, X_train_cat_tensor
    )  # Pass numeric and categorical tensors separately
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(
            f"Epoch {epoch+1}/{epochs:,.2f}, Loss: {loss.item():,.2f},"
            + f" lr: {optimizer.param_groups[0]['lr']:,.2f}"
        )
    if epoch % 25 == 0:
        scheduler.step(loss.item())

# Evaluate the model
model.eval()
with torch.no_grad():
    test_predictions = model(X_test_num_tensor, X_test_cat_tensor)
    mse = mean_squared_error(y_test_tensor, test_predictions)
    print(f"Mean Squared Error on Test Data: {mse}")

Epoch 1/1,000.00, Loss: nan, lr: 0.01
Epoch 101/1,000.00, Loss: nan, lr: 0.01
Epoch 201/1,000.00, Loss: nan, lr: 0.01
Epoch 301/1,000.00, Loss: nan, lr: 0.01
Epoch 401/1,000.00, Loss: nan, lr: 0.01
Epoch 501/1,000.00, Loss: nan, lr: 0.01
Epoch 601/1,000.00, Loss: nan, lr: 0.00
Epoch 701/1,000.00, Loss: nan, lr: 0.00
Epoch 801/1,000.00, Loss: nan, lr: 0.00
Epoch 901/1,000.00, Loss: nan, lr: 0.00


ValueError: Input contains NaN.

In [10]:
X_train_cat_tensor

tensor([[1, 2, 3],
        [4, 1, 3],
        [3, 4, 5],
        ...,
        [4, 2, 1],
        [3, 6, 2],
        [3, 2, 2]])

In [11]:
X_train_num_tensor

tensor([[ 2.5601, -2.5507,  2.9339,  2.2295,  2.1382,  1.7382],
        [ 0.4474, -1.2204,  1.1396,  0.7475,  0.6567,  0.5377],
        [ 0.6375,  0.5300,  0.2424,  0.7654,  0.7003,  0.7920],
        ...,
        [-0.9892, -1.0104,  0.2424, -1.1093, -1.1124, -1.1853],
        [ 0.2150,  0.7400,  0.6910,  0.3548,  0.2558,  0.3965],
        [ 0.7220, -0.9404,  0.2424,  0.9707,  0.9182,  0.8061]])

In [12]:
model.eval()
with torch.no_grad():
    test_predictions = model(X_test_num_tensor, X_test_cat_tensor)
    mse = mean_squared_error(y_test_tensor, test_predictions)
    print(f"Mean Squared Error on Test Data: {mse}")

Mean Squared Error on Test Data: 291539.15625


In [13]:
y_test_tensor

tensor([[ 559.],
        [2201.],
        [1238.],
        ...,
        [ 761.],
        [9836.],
        [3742.]])

In [14]:
test_predictions

tensor([[ 650.0410],
        [2434.5598],
        [1229.5330],
        ...,
        [ 712.3433],
        [9726.6943],
        [3760.2744]])

In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error

# ... (Your data preprocessing code remains the same) ...


# Define the neural network model with Transformer components
class DiamondPricePredictorWithTransformer(nn.Module):
    def __init__(self, num_input_dim, cat_embedding_sizes, hidden_dim):
        super(DiamondPricePredictorWithTransformer, self).__init__()

        # Embedding layers for categorical features
        self.embeddings = nn.ModuleList(
            [
                nn.Embedding(num_classes, emb_size)
                for num_classes, emb_size in cat_embedding_sizes
            ]
        )

        total_emb_dim = sum(emb_size for _, emb_size in cat_embedding_sizes)

        # Transformer Encoder Layer
        self.transformer_layer = nn.TransformerEncoderLayer(
            d_model=num_input_dim + total_emb_dim,
            nhead=4,  # Number of attention heads
            dim_feedforward=hidden_dim,
        )

        self.transformer_encoder = nn.TransformerEncoder(
            self.transformer_layer, num_layers=2
        )  # Number of transformer layers

        self.fc1 = nn.Linear(num_input_dim + total_emb_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)

        self.embedding_dim = 9  # 16  # Embedding dimension for column names
        self.column_name_embedding = nn.Embedding(
            len(X_train.columns), self.embedding_dim
        )

    def forward(self, num_inputs, cat_inputs, col_indices):
        # Embed column indices
        col_name_embeddings = self.column_name_embedding(col_indices)

        # Embed categorical features
        embeddings = [
            embedding(cat_inputs[:, i]) for i, embedding in enumerate(self.embeddings)
        ]
        cat_features = torch.cat(embeddings, dim=1)

        # Combine column embeddings with features
        x = torch.cat([col_name_embeddings, num_inputs, cat_features], dim=1)

        # Apply Transformer Encoder
        x = self.transformer_encoder(x)

        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Initialize the model
num_input_dim = X_train_num.shape[1]
cat_embedding_sizes = [
    (len(le.classes_), min(50, (len(le.classes_) + 1) // 2))
    for le in label_encoders.values()
]
hidden_dim = 64
model = DiamondPricePredictorWithTransformer(
    num_input_dim, cat_embedding_sizes, hidden_dim
)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ... (Training loop and evaluation code remains the same) ...

# During training loop and evaluation, make sure to provide the column indices as well
# Column indices can be obtained using: col_indices = torch.arange(len(X_train.columns))

AssertionError: embed_dim must be divisible by num_heads

In [5]:
# Define the neural network model with Transformer components
class DiamondPricePredictorWithTransformer(nn.Module):
    def __init__(self, num_input_dim, cat_embedding_sizes, hidden_dim, num_heads):
        super(DiamondPricePredictorWithTransformer, self).__init__()

        # Embedding layers for categorical features
        self.embeddings = nn.ModuleList(
            [
                nn.Embedding(num_classes, emb_size)
                for num_classes, emb_size in cat_embedding_sizes
            ]
        )

        total_emb_dim = sum(emb_size for _, emb_size in cat_embedding_sizes)

        # Calculate adjusted embedding dimension for compatibility with num_heads
        adjusted_embed_dim = num_heads * ((num_input_dim + total_emb_dim) // num_heads)

        # Transformer Encoder Layer
        self.transformer_layer = nn.TransformerEncoderLayer(
            d_model=adjusted_embed_dim,
            nhead=num_heads,
            dim_feedforward=hidden_dim,
        )

        self.transformer_encoder = nn.TransformerEncoder(
            self.transformer_layer, num_layers=2
        )  # Number of transformer layers

        self.fc1 = nn.Linear(adjusted_embed_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)

        self.embedding_dim = 16  # Embedding dimension for column names
        self.column_name_embedding = nn.Embedding(
            len(X_train.columns), self.embedding_dim
        )

    def forward(self, num_inputs, cat_inputs, col_indices):
        # Embed column indices
        col_name_embeddings = self.column_name_embedding(col_indices)

        # Embed categorical features
        embeddings = [
            embedding(cat_inputs[:, i]) for i, embedding in enumerate(self.embeddings)
        ]
        cat_features = torch.cat(embeddings, dim=1)

        # Expand dimensions to make room for the col_name_embeddings
        col_name_embeddings_expanded = col_name_embeddings.unsqueeze(0).expand(
            num_inputs.size(0), -1, -1
        )

        # Expand dimensions for num_inputs and cat_features to match col_name_embeddings
        num_inputs_expanded = num_inputs.unsqueeze(1).expand(
            -1, col_name_embeddings_expanded.size(1), -1
        )
        cat_features_expanded = cat_features.unsqueeze(1).expand(
            -1, col_name_embeddings_expanded.size(1), -1
        )

        # Concatenate the embeddings and features
        x = torch.cat(
            [col_name_embeddings_expanded, num_inputs_expanded, cat_features_expanded],
            dim=2,
        )

        # Permute dimensions to match transformer's expected input shape
        x = x.permute(1, 0, 2)

        # Apply Transformer Encoder
        x = self.transformer_encoder(x)

        # Permute dimensions back to original shape
        x = x.permute(1, 0, 2)

        # Flatten back the input dimensions
        x = x.view(x.size(0), -1)

        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Initialize the model
num_input_dim = X_train_num.shape[1]
cat_embedding_sizes = [
    (len(le.classes_), min(50, (len(le.classes_) + 1) // 2))
    for le in label_encoders.values()
]
hidden_dim = 64
num_heads = 4  # Choose a suitable number of attention heads
model = DiamondPricePredictorWithTransformer(
    num_input_dim, cat_embedding_sizes, hidden_dim, num_heads
)

epochs = 10000
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Training loop
col_indices = torch.arange(len(X_train.columns))
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(
        X_train_num_tensor, X_train_cat_tensor, col_indices
    )  # Pass numeric, categorical tensors, and column indices
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

# Evaluation
with torch.no_grad():
    test_predictions = model(
        X_test_num_tensor, X_test_cat_tensor, col_indices
    )  # Pass numeric, categorical tensors, and column indices
    mse = mean_squared_error(y_test_tensor, test_predictions)
    print(f"Mean Squared Error on Test Data: {mse}")

AssertionError: was expecting embedding dimension of 16, but got 33